# **Compliance GPT with LangChain**

## **Setup**

In [1]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config
from utils.models import ModelName, get_model

config = get_config()

In [3]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [4]:
model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config)

## **Load Vector Store Index**

In [5]:
from databases.vector_store import RedisIndexManager

redis = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
vector_store = redis.load_vector_index()

## **Get Retriever**

In [6]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk

retriever_ojk = get_retriever_ojk(vector_store=vector_store, top_n=6, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)

In [7]:
from retriever.retriever_sikepo.lotr_sikepo import lotr_sikepo

redis_sikepo_ket = RedisIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config, db_id=0)
vector_store_ket = redis_sikepo_ket.load_vector_index()
retriever_sikepo_ket = lotr_sikepo(vector_store=vector_store_ket, llm_model=llm_model, embed_model=embed_model, config=config)

redis_sikepo_rek = RedisIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config, db_id=0)
vector_store_rek = redis_sikepo_rek.load_vector_index()
retriever_sikepo_rek = lotr_sikepo(vector_store=vector_store_rek, llm_model=llm_model, embed_model=embed_model, config=config)

## **Create Chain**

In [8]:
# from constant.ojk.prompt import CONTEXTUALIZE_Q_PROMPT_STR, QA_SYSTEM_PROMPT_STR
# from databases.chat_store import MongoDBChatStore, RedisChatStore
# from chain.chain_ojk.chain_ojk import create_ojk_chain
# from chain.rag_chain import create_chain_with_chat_history

# chat_store = RedisChatStore(k=4, config=config, db_id=1)

# chain = create_ojk_chain(
#     contextualize_q_prompt_str=CONTEXTUALIZE_Q_PROMPT_STR,
#     qa_system_prompt_str=QA_SYSTEM_PROMPT_STR,
#     retriever=retriever,
#     llm_model=llm_model,
#     # chat_store=chat_store
# )

# chain_history = create_chain_with_chat_history(
#     final_chain=chain,
#     chat_store=chat_store
# )

In [9]:
import os
from databases.chat_store import RedisChatStore
from chain.rag_chain import create_chain_with_chat_history, create_chain
from retriever.retriever_sikepo.graph_cypher_retriever import graph_rag_chain
from langchain_community.graphs import Neo4jGraph


URL = os.getenv("NEO4J_GRAPH_URL")
USERNAME = os.getenv("NEO4J_USERNAME")
PASSWORD = os.getenv("NEO4J_PASSWORD")

print(URL)

graph = Neo4jGraph(url=URL, username=USERNAME, password= PASSWORD)
graph_chain = graph_rag_chain(llm_model, llm_model, graph)

chat_store = RedisChatStore(k=4, config=config, db_id=1)

chain = create_chain(retriever_ojk, retriever_sikepo_rek, retriever_sikepo_ket, graph_chain, llm_model)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store
)

bolt://108.137.122.156:7687


TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

## **Invoke**

#### **With Question, Answer, and Context**

In [24]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="BErapa SWDKLLJ dari buldozer?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

KeyboardInterrupt: 

In [19]:
print(response['answer'])

SWDKLLJ untuk sedan tidak ditemukan dalam konteks dan informasi metadata yang diberikan.


In [16]:
print(response["context"])

[
  {
    "metadata": {
      "id": "doc:ojk:0d063bb180c64673a070b97e6d024c25",
      "title": "Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan",
      "sector": "IKNB",
      "subsector": "Asuransi",
      "regulation_type": "Klasifikasi Bapepam",
      "regulation_number": "36/PMK.010/2008",
      "effective_date": "26 Februari 2008",
      "file_url": "https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf",
      "doc_id": "1718",
      "page_number": "3",
      "relevance_score": 0.2600135
    },
    "page_content": "MENTERI KEUANGAN\nREPUBLIK INDONESIA\nmemperoleh santunan sebesar Rp 25.000.000,00 (dua puluh lima\njuta rupiah).\nb. Korban yang mengalami cacat tetap berhak memperoleh\nsantunan yang besarnya dihitung berdasarkan angka prosentase\n Korban yang memerlukan perawatan d

#### **Streaming**

In [19]:
from chain.rag_chain import print_answer_stream

print_answer_stream(chain=chain_history, question="Siapa itu lionel messi?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Saya minta maaf, tetapi pertanyaan yang Anda ajukan tidak terkait dengan konteks dan informasi metadata yang diberikan. Silakan ajukan pertanyaan lain yang terkait dengan kepatuhan perbankan di Indonesia.

#### **Chat Store Monitor**

In [44]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Apa judul peraturan dengan nomor regulasi 3/SEOJK.03/2021?
AI: Pelaporan dan Permintaan Informasi Debitur Melalui Sistem Layanan Informasi Keuangan (SEOJK SLIK) 

Sumber: [3/SEOJK.03/2021](https://www.ojk.go.id/id/regulasi/Documents/Pages/Pelaporan-dan-Permintaan-Informasi-Debitur-Melalui-Sistem-Layanan-Informasi-Keuangan--/SEOJK%20SLIK%2003-03-2021.pdf)
Human: Apa judul peraturan dengan nomor regulasi 5 /SEOJK.05/2019?
AI: Bentuk, Susunan, dan Tata Cara Penyampaian Laporan Berkala bagi Pelaku Usaha Pergadaian dan Perusahaan Pergadaian

Source: [5/SEOJK.05/2019](https://www.ojk.go.id/id/regulasi/Documents/Pages/Bentuk,-Susunan,-dan-Tata-Cara-Penyampaian-Laporan-Berkala-bagi-Pelaku-Usaha-Pergadaian-dan-Perusahaan-Pergadaian/seojk%205-2019.pdf)
Human: Berikan Petunjuk Pengisian Format Laporan Profil Perusahaan Pergadaian Swasta Syariah?
AI: Format laporan profil perusahaan pergadaian swasta syariah diisi dengan informasi yang belum dimuat dan/atau tidak sama dengan laporan perusah

In [31]:
# to make the chat history empty
chat_store.clear_all()